
# Flask App - Clinical Trial Eligiblity AI

In [1]:
import joblib
import openai
from langchain.chat_models import ChatOpenAI
import os
import string  
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter
import numpy as np
import nltk
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from skmultilearn.problem_transform import ClassifierChain
from gensim.models import Word2Vec
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from langchain.text_splitter import TokenTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
# pre-trained classifier model
classifier = joblib.load('SVC1.pkl')

## Required Methods

In [3]:
""" To clean input text"""

def clean_text(text):
    # Download necessary NLTK resources if not already available
    nltk.download('punkt')  # Download tokenizer
    nltk.download('stopwords')  # Download stopwords
    nltk.download('wordnet')  # Download WordNet for lemmatization
    # Step 1: Remove newline/tab patterns, numbers longer than 3 (optional)
    pattern = r"[\n\t]\d+"  # Capture newline/tab followed by digits
    text = re.sub(pattern, "", text)
    pattern = r'\b\d{4,}\b'  # Match numbers of length greater than 3
    text = re.sub(pattern, '', text)
    # Step 2: Remove special characters
    pattern = r'[^a-zA-Z0-9\s\-_]'  # Match non-alphanumeric, whitespace, hyphens, or underscores
    text = re.sub(pattern, '', text)
    # Step 3: Convert to lowercase
    text = text.lower()
    # Step 4: Tokenize
    tokens = word_tokenize(text)
    # Step 5: Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Step 6: Lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    # Step 7: Join tokens back into a string
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

""" Generate features"""

def tfidf_weighted_average(document, word_embeddings, tfidf_weights, default_embedding=None):

    word_counts = Counter(document.split())
    total_words = len(word_counts)  # Count of unique words
    if default_embedding is None:
        default_embedding = np.zeros_like(next(iter(word_embeddings.values())))  # Use the first embedding vector as the default
    vocabulary = set(word_counts.keys())
    if set(word_embeddings.keys()) != vocabulary:
        raise ValueError("Vocabulary of word_embeddings doesn't match the vocabulary of the document")
    # Calculate weighted sum of word embeddings (handling missing words and efficiency)
    weighted_sum = np.zeros_like(default_embedding)
    for word_dict in tfidf_weights:
        word = word_dict.get('word', None)
        weight = word_dict.get('weight', 0)
        if word and word in word_embeddings:
            weighted_sum += weight * word_embeddings[word]
    # Calculate mean of weighted word embeddings
    average_embedding = weighted_sum / total_words
    return average_embedding
from gensim.models import Word2Vec

def tfidf_weights_input(documents):
    tfidf_weights = []
    word_counts = Counter(documents.split())
    total_words = len(word_counts)

    # Calculate IDF weights
    idf_weights = {word: np.log(total_words / word_counts.get(word, 1)) for word in word_counts}

    # Calculate TF-IDF weights
    tfidf_weights.append({word: (count / total_words) * idf_weights[word] for word, count in word_counts.items()})

    return tfidf_weights

def generate_word_embedding(text):
    # Tokenize the text into words
    tokenized_text = text.split()
    # Initialize and train the Word2Vec model
    model = Word2Vec([tokenized_text], min_count=1, vector_size=100)  
    # Generate word embeddings for the current text using the trained model
    word_embeddings = {word: model.wv[word] for word in tokenized_text}
    return word_embeddings

def generate_features(preprocessed_text,weighted_embeddings):
    tfidf_vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))  # Consider bigrams for richer features
    tfidf_features = tfidf_vectorizer.fit_transform(preprocessed_text).astype(np.float64)
    X_weighted_embedding = np.array(weighted_embeddings).astype(np.float64)
    X = hstack((X_weighted_embedding, tfidf_features))
    return X

"""Decodes the classifier output"""

def decode_condition_array(data_array):
    label_cols = ['ABDOMINAL', 'CREATININE', 'MAJOR-DIABETES', 'ADVANCED-CAD']
    # Convert array to list and get first row
    data_list = data_array.tolist()[0]
    # Create a dictionary for mapping 
    label_mapping = {col: 0 for col in label_cols}

    # Decode the list based on mapping
    decoded_list = []
    for i, value in enumerate(data_list):
        if value == 0:
            decoded_list.append(label_cols[i])  # Append label if condition met
    return decoded_list


"""  Data preprocessing for summarization"""

def chunk_data(text):
    char_text_splitter = CharacterTextSplitter(separator="\n", chunk_size=4000,
                                               chunk_overlap=10, length_function=len)
    text_chunks = char_text_splitter.split_text(text)
    return text_chunks


def generate_tokens(s):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_text(s)
    return text_splitter.create_documents(splits)

### Application and Abstarctive Summarization

In [9]:
from flask import Flask, render_template, request
import joblib
from langchain.chains.summarize import load_summarize_chain
from langchain import OpenAI
from langchain import PromptTemplate
import textwrap
from time import monotonic

app = Flask(__name__)

# Load the machine learning model
#classifier = joblib.load('SVM.pkl')

@app.route('/', methods=['GET', 'POST'])
def index():
    summary = None
    classification_result = None
    empty_classification_result = None
    message=None
    if request.method == 'POST':
        text = request.form['text']
        
        #print(text)
        #print(type(text))
        summary, classification_result,message = generate_summary_and_classification(text)
        if((classification_result is None ) or (len(classification_result) <= 0) or (classification_result[0] == "")) :
            empty_classification_result = "This sample is not eligible for the clinical trial"
    return render_template('homepage.html', summary=summary , classification_result=classification_result , message = message , empty_classification_result = empty_classification_result )

def generate_summary_and_classification(text):
    
    summary = "This is a sample summary."
    #text_chunks= " Record date: 2067-12-06 Chief Complaint 1. cough is better. She uses the inhalers regularly. 2. Funny taste in the mouth after she does the inhalers in the morning. No sore throat. She does rinse her mouth after the pulmicort. 3. No recurrence of the hand numbness. Problems Hypertension Insomnia H/O cerebrovascular accident borderline aodm Basal cell carcinoma Medications Vioxx (ROFECOXIB) 25MG, 1 Tablet(s) PO QD Lotrisone Cream (BETAMETHASONE And Clotri... APPLY CREAM TOP BID Asa (ACETYLSALICYLIC Acid) 325MG, 1 Tablet(s) PO QD Allegra (FEXOFENADINE Hcl) 60MG, 1 Tablet(s) PO BID PRN . Lipitor (ATORVASTATIN) 10MG, 1 Tablet(s) PO QD ATENOLOL 100MG, 1 Tablet(s) PO BID : take 1/2 in morning and 1 in evening Flonase (FLUTICASONE Nasal Spray) 1-2 SPRAY NAS QD Celexa (CITALOPRAM) 20MG, 1 Tablet(s) PO QD Prilosec (OMEPRAZOLE) 20MG, 1 Capsule(s) PO QHS Dyazide (TRIAMTERENE 37.5 Mg/hctz 25 Mg) 1 CAPSULE PO QD LISINOPRIL 10MG, 1 Tablet(s) PO QD Albuterol Inhaler 2 PUFF INH Q4-6H PRN .cough : use with chamber Singulair (MONTELUKAST) 10MG, 1 Tablet(s) PO QHS Pulmicort (BUDESONIDE Oral Inhaler) 2 PUFF INH BID Plavix (CLOPIDOGREL) 75 MG PO QD Allergies Sulfa - Unknown Penicillins - Unknown Social History Married. Her husband has Alzheimer's. His moods are variable. She has been going to FitSport and exercising better. non-smoker. Six children and six grandchildren. No alcohol in 30 years. Prior alcohol dependence. Her daughter, son-in-law and 18 month old twins are living in her house. Her Daughter, Uber, in Vietnam may come back for the holidays. Her Daughter, Ingalls, lives in BOURNE-she is a variable cause of stress. She is going to FitSport almost every day. Review of Systems ROS-Sleep has been OK, occ wakening in the middle of the night-she uses amytriptiline at night. energy is better since she is exercising more. Vital Signs BLOOD PRESSURE 140/80 Exam HEENT-NC/AT, EOMI, PERRL TM-Clear Pharynx-Clear Chest-clear to P and A Cor-S1S2 normal no murmers, rubs or gallops Abd-soft, nontender, no hepatosplenomegaly Ext-no edema Disposition and Plans 1. Cough-we discussed that she will need to stay on the albuterol and the pulmicort in order to prevent the cough from coming back. 2. Insomnia which had been such a problem seems better now. Continue the celexa 20mg and she is also on amytriptiline (per Dr. Dill). She will double check the dose through him. 3. Hyperlipidemia-On lipitor. Check LFTs and CPK along with chol/HDL today. 4. HTN-good control on the lisinopril, atenolol and dyazide. Check electrolytes today. 5. Borderline AODM. Check glucose today. ______________________________ Othel Vidal, M.D. **************************************************************************************************** Record date: 2068-06-01 Chief Complaint 1. She increased the lisinopril to 20mg. she is tolerating it well. No dizziness, no side effects. BP at home is excellent. 2. She is interested in starting an antioxidant, her friend used it and felt better. 3. Skin lesions: left side of the face, left forearm. Problems Hypertension Insomnia H/O cerebrovascular accident borderline aodm Basal cell carcinoma Medications Vioxx (ROFECOXIB) 25MG TABLET take 1 Tablet(s) PO QD Lotrisone Cream (BETAMETHASONE And Clotri... APPLY CREAM TOP BID Asa (ACETYLSALICYLIC Acid) 325MG TABLET take 1 Tablet(s) PO QD Allegra (FEXOFENADINE Hcl) 60MG CAPSULE take 1 Tablet(s) PO BID PRN Lipitor (ATORVASTATIN) 10MG TABLET take 1 Tablet(s) PO QD ATENOLOL 100MG TABLET take 1 Tablet(s) PO BID , take 1/2 in morning and 1 in evening Flonase (FLUTICASONE Nasal Spray) 1-2 SPRAY NAS QD Prilosec (OMEPRAZOLE) 20MG CAPSULE CR take 1 Capsule(s) PO QHS Dyazide (TRIAMTERENE 37.5 Mg/hctz 25 Mg) 1 CAPSULE PO QD Albuterol Inhaler 2 PUFF INH Q4-6H PRN .cough , use with chamber Singulair (MONTELUKAST) 10MG TABLET take 1 Tablet(s) PO QHS Plavix (CLOPIDOGREL) 75 MG PO QD Pulmicort (BUDESONIDE Oral Inhaler) 4 PUFF INH BID Robitussin Ac (GUAIFENESIN Ac) 5 ML TO 10 ML PO Q4H PRN .cough , UL5104263 Celexa (CITALOPRAM) 20MG TABLET take 1 Tablet(s) PO QD LISINOPRIL 20MG TABLET take 1 PO QD Allergies Sulfa - Unknown Penicillins - Unknown Social History Married. Her husband has Alzheimer's. He has gotten a little more 'mellow' non-smoker. Six children and six grandchildren. Her daughter with her 2 year old twins, and other daughter, Judd, are living with her. No alcohol in 30 years. Prior alcohol dependence. Review of Systems ROS-no chest pain, shortness of breath, no cough. Mood is good. Sleep is good. Vital Signs BLOOD PRESSURE 114/64 Exam Chest-clear to P and A Cor-S1S2 normal no murmers, rubs or gallops Abd-soft, nontender, no hepatosplenomegaly Ext-no edema Skin-raised, erythematous lesion near left angle of the jaw, raised verrucous lestion on left calf. Disposition and Plans 1. BP is under excellent control. Continue atenolol, dyzide, lisinopril. f/u with me in 3 months, sooner for any problems. 2. Asthma-under good control. 3. Skin lesions: she will f/u with Dr. Kim. ______________________________ Othel Vidal, M.D. **************************************************************************************************** Record date: 2068-09-26 Reason for Visit Patient presents for pre-op physical exam. Having left eye cataract surgery Dr. Janes on 10/04/68. Feels well. Problems Hypertension Insomnia H/O cerebrovascular accident borderline aodm Basal cell carcinoma Asthma Hyperlipidemia Procedures Tonsillectomy Medications Asa (ACETYLSALICYLIC Acid) 325MG TABLET take 1 Tablet(s) PO QD Allegra (FEXOFENADINE Hcl) 60MG CAPSULE take 1 Tablet(s) PO BID PRN ATENOLOL 100MG TABLET take 1 Tablet(s) PO BID , take 1/2 in morning and 1 in evening Flonase (FLUTICASONE Nasal Spray) 1-2 SPRAY NAS QD Prilosec (OMEPRAZOLE) 20MG CAPSULE CR take 1 Capsule(s) PO QHS Dyazide (TRIAMTERENE 37.5 Mg/hctz 25 Mg) 1 CAPSULE PO QD Albuterol Inhaler 2 PUFF INH Q4-6H PRN .cough , use with chamber Plavix (CLOPIDOGREL) 75 MG PO QD Pulmicort (BUDESONIDE Oral Inhaler) 4 PUFF INH BID Celexa (CITALOPRAM) 20MG TABLET take 1 Tablet(s) PO QD LISINOPRIL 20MG TABLET take 1 PO QD Lipitor (ATORVASTATIN) 10MG TABLET take 1 Tablet(s) PO QD Singulair (MONTELUKAST) 10MG TABLET take 1 Tablet(s) PO QHS Allergies Sulfa - Unknown Penicillins - Unknown Assessment and Plan 1. Pre-op Physical Exam -EKG -Labs: CBC, PT, PTT -Completed form, see chart -will fax form once have lab results _____________________________________________ Inez Yelverton, NP **************************************************************************************************** Record date: 2070-04-02 Chief Complaint 1. Feels pretty well. She had the MOHS surgery on the left side of her face, it has gotten infected. 2. Her husband, Xin, has progressive dementia and that is difficult. Problems Hypertension on atenolol and lisinopril Insomnia much better recently H/O cerebrovascular accident on aspirin and plavix Basal cell carcinoma s/p MOHS surgery in 3/2070 Asthma well controlled with flovent and singulair. She uses PRN albuterol Hyperlipidemia on Lipitor Diabetes mellitus type 2 : Started metformin 2069 checking blood sugars at home. Medications Asa (ACETYLSALICYLIC Acid) 325MG TABLET take 1 Tablet(s) PO QD Flonase (FLUTICASONE Nasal Spray) 1-2 SPRAY NAS QD Prilosec (OMEPRAZOLE) 20MG CAPSULE CR take 1 Capsule(s) PO QHS Albuterol Inhaler 2 PUFF INH Q4-6H PRN .cough , use with chamber Flovent (FLUTICASONE Propionate) 44MCG AEROSOL take 2 Puff(s) INH BID Dyazide (TRIAMTERENE 37.5 Mg/hctz 25 Mg) 1 CAPSULE PO QD Plavix (CLOPIDOGREL) 75 MG PO QD Lipitor (ATORVASTATIN) 10MG TABLET take 1 Tablet(s) PO QD Glucophage (METFORMIN) 500MG TABLET PO BID ATENOLOL 100MG TABLET take 1 Tablet(s) PO BID , take 1/2 in morning and 1 in evening Celexa (CITALOPRAM) 20MG TABLET take 1 Tablet(s) PO QD LISINOPRIL 20MG TABLET take 1 PO QD Singulair (MONTELUKAST) 10MG TABLET take 1 Tablet(s) PO QHS Allegra (FEXOFENADINE Hcl) 60MG CAPSULE take 1 Tablet(s) PO BID PRN Allergies Sulfa - Unknown Penicillins - Unknown Social History Marital status: married EtOH: none for many years Occupation: cashier, active in the community Exercise: active liefstyle Review of Systems ROS-no fever, chills, shortness of breath. She has ongoing cough. Vital Signs BLOOD PRESSURE 120/78 Exam Chest-clear to P and A Cor-S1S2 normal no murmers, rubs or gallops Abd-soft, nontender, no hepatosplenomegaly Ext-no edema Disposition and Plans 1. Routine health-scheduled mammogram, colonoscopy. 2. Type 2 DM-she will see Carolyn Delong, nutrition. She will start doing home glucose monitoring. Continue the metformin BID. Check HbA1C today, she will schedule ophtho and podiatry apt for yearly check. 3. HTN is under excellent control. 4. Depression is controlled with celexa-she is sleeping better. 5. Asthma-controlled with singulair, flovent and albuterol. 6. Hyperlipidemia-check labs every 6 months. ______________________________ Othel Vidal, M.D. **************************************************************************************************** Record date: 2071-11-17 Chief Complaint Urso comes in with several days of: nausea, diarrhea (5-6 times) with 'everything going through her'. She has no vomiting. + abd cramping pain. No blood in the stool. She had low grade fever. In addition, she feels increased chest congestion, cough, productive of white sputum. Her finger sticks at home are running at 280. No one at home has been sick. She had dental work done and received antibiotics about a month ago. Problems Hypertension : Well controlled on dyazide, lisinopril and atenolol Insomnia H/O cerebrovascular accident Basal cell carcinoma Asthma : on Flovent and Singulair. Good control Hyperlipidemia : On lipitor Diabetes mellitus type 2 : On Glucovance 2.5/500 BID Medications Albuterol Inhaler 2 PUFF INH Q4-6H PRN .cough , use with chamber ATENOLOL 100MG TABLET take 1 Tablet(s) PO BID , take 1/2 in morning and 1 in evening Carmol 40 Cream (UREA 40%) 1 APPLICATION TOP QD , apply to thick skin on feet once a day Celexa (CITALOPRAM) 20MG TABLET take 1 Tablet(s) PO QD Dyazide (TRIAMTERENE 37.5 Mg/hctz 25 Mg) 1 CAPSULE PO QD Flonase (FLUTICASONE Nasal Spray) 1-2 SPRAY NAS QD Flovent (FLUTICASONE Propionate) 44MCG AEROSOL take 2 Puff(s) INH BID Glucovance (GLYBURIDE/METFORMIN) 2.5/500 MG PO BID Lipitor (ATORVASTATIN) 10MG TABLET take 1 Tablet(s) PO QD LISINOPRIL 20MG TABLET take 1 PO QD Plavix (CLOPIDOGREL) 75 MG PO QD PREDNISONE 10MG TABLET take 4 po qd x 3 days, 2 po qd x 3 days, 1 po qd x 3 days Tablet(s) PO QAM PRN bronchitis Serevent Diskus (SALMETEROL Diskus) 1 PUFF INH BID Singulair (MONTELUKAST) 10MG TABLET take 1 Tablet(s) PO QHS Triamcinolone Ointment 0.1% OINTMENT TOP bid PRN , apply to rash two to three times per day; avoid face, groin, and underarms Allergies Sulfa - Unknown Penicillins - Unknown Social History Married, her husband has Alzheimers. He has been moved to Portland Sanatorium. Her daughter and 3 grandchildren are living with her. Her other daughter has breast cancer and is going through RT treatments. She had 7 children, one died. She has 6 adult children. EtOH-none Smoking-none Review of Systems ROS-No chest pain, no headache, no photophobia. Vital Signs BLOOD PRESSURE 110/68 PULSE 83 TEMPERATURE 98.1 F O2 SAT 96 Exam HEENT-NC/AT, EOMI, PERRL TM-Clear Pharynx-Clear Chest-clear to P and A-no wheezing on exam today. Cor-S1S2 normal no murmers, rubs or gallops Abd-soft, nontender, no hepatosplenomegaly Disposition and Plans 1. Type 2 DM. She has elevated blood sugar at home. Will increase glyburide/metformin to 5/500. Check HbA1c today. 2. Diarrhea-will check C.diff and stool C and S. Use imodium PRN. 3. Cough and congestion. Will check CXR today. Will write for zithromax, hold on starting and call tomorrow with how she is doing. 4. Chronic cough and mucus production. Will f/u again in 10 days, bring in medication that she is using and discuss further management. ______________________________ Othel Vidal, M.D."
    cleaned_text = clean_text(text)
    weight_embeddings=tfidf_weighted_average(cleaned_text, generate_word_embedding(cleaned_text), tfidf_weights_input(cleaned_text), default_embedding=None)
    

    try :
        X = generate_features([cleaned_text],weight_embeddings)
        prediction = classifier.predict(X)
        predictions_n = prediction.toarray() 
        decoded_list = decode_condition_array(predictions_n)
        text= generate_tokens(cleaned_text)
    except :
        message = "Enter the patient record data"
        print("alert sent for the patient record data")
        return ["",[],message]
        
#         print("X")
#         print(X)
#         print(type(X))
        
    
    
    print("decoded_list")
    print(decoded_list)
    print("predictions_n")
    print(predictions_n)
    
    question_prompt = PromptTemplate.from_template(  '''
        You need to summarize the patients clinical data {text} focusing on the key factors that make this patient a strong candidate for the clinical trial 
        You can use the following points 
        Chief Complaint:
        Patient Information:
        Medical History:
        Why he can be considered to clinical trail: 

      ''')
    #decoded_list = ['CREATININE']

    refine_prompt = PromptTemplate.from_template(
      f"""
      Your job is to produce a summary based on the Classification output explaining why the patient can be selected for : {', '.join(decoded_list)}.
      """
      "{text}"
    )


    llm = OpenAI(openai_api_key="")

    chain = load_summarize_chain(
        llm, 
        chain_type="refine", 
        question_prompt=question_prompt,
        refine_prompt = refine_prompt, 
        document_variable_name="text"
    )
    
    # Remember to define start_time before running the chain to calculate runtime
    start_time = monotonic()

    summary = chain.run(text)  # You can still use text_chunks here for other purposes

    print(f"Summary: {textwrap.fill(summary, width=100)}")
    print(f"Chain type: {chain.__class__.__name__}")
    print(f"Run time: {monotonic() - start_time}")
    return [summary,decoded_list,""]

if __name__ == '__main__':
    app.run( port=8089)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8089
Press CTRL+C to quit
127.0.0.1 - - [05/May/2024 00:03:00] "GET / HTTP/1.1" 200 -
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
127.0.0.1 - - [05/May/2024 00:03:51] "POST / HTTP/1.1" 200 -


alert sent for the patient record data


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aslam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


decoded_list
['CREATININE', 'MAJOR-DIABETES', 'ADVANCED-CAD']
predictions_n
[[1. 0. 0. 0.]]


127.0.0.1 - - [05/May/2024 00:06:30] "POST / HTTP/1.1" 200 -


Summary:         This patient can be selected for CREATININE because they have ESRD, or end-stage renal
disease, and are using a tunneled HD line for dialysis. They also have a nephrologist following
their care and have a normal HCV viral load, making them a good candidate for further evaluation and
treatment.   The patient can also be selected for MAJOR-DIABETES due to their DM2, or type 2
diabetes, and high A1C levels. They are currently on a regimen for their diabetes and have recently
completed a sleep study that qualifies them for CPAP therapy.   Lastly, the patient can be selected
for ADVANCED-CAD due to their cardiac regimen, history of cardiac issues, and recent suture
bleeding. They are also being followed by a cardiologist and have a normal liver function, making
them a good candidate for further evaluation and treatment for their advanced cardiac disease.
Chain type: RefineDocumentsChain
Run time: 35.78099999995902
